<a href="https://colab.research.google.com/github/raven-gith/machinelearning1/blob/main/14.%20Chapter%2014/chapter_14_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 14: Transfer Learning with Pretrained Models

Notebook ini mereproduksi dan menjelaskan isi Bab 14 dari buku _Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow_ oleh Aurélien Géron.

## 📘 Ringkasan Materi:

Bab ini menjelaskan cara memanfaatkan **Transfer Learning** dengan **pre-trained model** untuk tugas klasifikasi citra.

### Materi utama:
1. Load pre-trained model dari `keras.applications`
2. Freeze layer awal, latih hanya classifier
3. Fine-tuning seluruh model
4. Preprocessing gambar dengan pipeline standar


In [1]:

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import MobileNetV2

# Load base model tanpa classifier top
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(96, 96, 3))
base_model.trainable = False  # freeze semua layer

# Tambahkan classifier head
model = keras.Sequential([
    base_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(1, activation="sigmoid")
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_96             │ (None, 3, 3, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [2]:

import numpy as np

# Simulasi data gambar 100 sampel (2 kelas)
X_train = np.random.rand(100, 96, 96, 3)
y_train = np.random.randint(0, 2, size=100)

# Train classifier head (transfer learning)
model.fit(X_train, y_train, epochs=3)


Epoch 1/3
4/4 ━━━━━━━━━━━━━━━━━━━━ 14s 356ms/step - accuracy: 0.5158 - loss: 0.7238
Epoch 2/3
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 197ms/step - accuracy: 0.5170 - loss: 0.7250
Epoch 3/3
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 188ms/step - accuracy: 0.5151 - loss: 0.7620


In [3]:

# Unfreeze base model
base_model.trainable = True

# Re-compile dan fine-tune
model.compile(optimizer=keras.optimizers.Adam(1e-5),  # learning rate kecil
              loss="binary_crossentropy",
              metrics=["accuracy"])

model.fit(X_train, y_train, epochs=3)


Epoch 1/3
4/4 ━━━━━━━━━━━━━━━━━━━━ 58s 1s/step - accuracy: 0.5361 - loss: 0.9294
Epoch 2/3
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.5828 - loss: 0.7388
Epoch 3/3
4/4 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.5995 - loss: 0.7209
